#Imports

In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Create Fully connected Network

In [67]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50,num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

# Create Simple CNN

In [68]:
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) #same convolution
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)) #same convolution
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x


# Saving The Trained Model

In [69]:
def save_checkpoint(state, filename = "my_checkpoint.pth.tar"):
  print("=> Saving checkpoint")
  torch.save(state, filename)

# Getting The Saved Model

In [70]:
def load_checkpoint(checkpoint):
  print("=> Loading checkpoint")
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])

In [71]:
# model = NN(784,10)
# x = torch.randn(64,784)
# print(model(x).shape)

# Set Device

In [72]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [73]:
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 15
load_model = True

# Load Data

In [74]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Initialize network

In [75]:
model = CNN().to(device)

# Loss and optimizer

In [76]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [77]:
if load_model:
  load_checkpoint(torch.load("my_checkpoint.pth.tar"))

=> Loading checkpoint


# Train Network

In [78]:
for epoch in range(num_epochs):
  
  losses = []

  if epoch%3==0:
    checkpoint = {'state_dict' : model.state_dict(), 'optimizer': optimizer.state_dict()}
    save_checkpoint(checkpoint)

  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)
    losses.append(loss.item())

    # backward
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent or adam step
    optimizer.step()
  mean_loss = sum(losses)/len(losses)
  print(f'Loss at epoch {epoch} was {mean_loss:.5f}')


=> Saving checkpoint
Loss at epoch 0 was 0.02547
Loss at epoch 1 was 0.02428
Loss at epoch 2 was 0.02230
=> Saving checkpoint
Loss at epoch 3 was 0.02021
Loss at epoch 4 was 0.01922
Loss at epoch 5 was 0.01808
=> Saving checkpoint
Loss at epoch 6 was 0.01644
Loss at epoch 7 was 0.01544
Loss at epoch 8 was 0.01481
=> Saving checkpoint
Loss at epoch 9 was 0.01343
Loss at epoch 10 was 0.01335
Loss at epoch 11 was 0.01214
=> Saving checkpoint
Loss at epoch 12 was 0.01168
Loss at epoch 13 was 0.01114
Loss at epoch 14 was 0.00954


#Check accuracy on training and test to see how good our model

In [79]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct+=(predictions==y).sum()
      num_samples+=predictions.size(0)

    print(f'Got {num_correct/num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
  # model.train()



# Check Accuracy

In [80]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 0.9975500106811523 with accuracy 99.76
Checking accuracy on training data
Got 0.9975500106811523 with accuracy 99.76
